### Lean Converter
Takes the input as a csv file for a ticker, then converts it to the LEAN CLI format. The output is a csv file that can be used to import the data into LEAN.

In [1]:
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import zipfile
import os

def convert_to_lean_format(csv_file, ticker, frequency='daily'):
    '''
    Converts a CSV file containing stock data into a format compatible with LEAN Local CLI Framework.
    Args:
        csv_file (str): The path to the input CSV file containing stock data.
        ticker (str): The stock ticker symbol.
        frequency (str, optional): The frequency of the data. Can be 'daily', 'hourly', or 'minute'. Defaults to 'daily'.
    Returns:
        None: The function saves the converted data to a file in the appropriate directory based on the frequency. Located within project directory.
    '''
    # Read the Loaded CSV File
    df = pd.read_csv(csv_file)
    
    # Convert the time to NY time (exchange time zone) Optional, still deciding if needed
    #convert_utc_to_ny(df, datetime_column='ts_event')
    
    # Strip the extra time info from CONVERTED time zone to match lean format
    df['date'] = pd.to_datetime(df.pop('ts_event')).dt.strftime('%Y%m%d %H:%M')
    df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
    
    # Conversion from dollars to deci-cents
    df[['open', 'high', 'low', 'close']] = (df[['open', 'high', 'low', 'close']] * 10000).astype(int)
    
    # Data type sorting for directory saving for csv
    if frequency == 'daily':
        output_dir = 'data/equity/usa/daily/'
        output_file = f"{output_dir}{ticker.lower()}.csv"
    elif frequency == 'hourly':
        output_dir = 'data/equity/usa/hourly/'
        output_file = f"{output_dir}{ticker.lower()}.csv"
    else:
        output_dir = 'data/equity/usa/minute/'
        output_file = f"{output_dir}{(ticker.lower())}.csv"
    
    df.to_csv(output_file, index=False, header=False)
        
    # Zip File
    zip_file = os.path.join(output_dir, f'{ticker.lower()}.zip')
    # Create a zip file and add the csv file to it
    with zipfile.ZipFile(zip_file, 'w') as zf:
        zf.write(output_file, arcname=f'{ticker.lower()}.csv')

    # Optionally, remove the csv file after zipping (if you want the zip to be the only output)
    #os.remove(csv_file)

    print(f"{output_file} has been successfully zipped into {zip_file}.")